In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

task='emotion'
MODEL = "cardiffnlp/twitter-roberta-base-emotion"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [2]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [3]:
# download label mapping
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

In [4]:
import pandas as pd
df_tweet = pd.read_parquet('../../../covid_media_coverage/data/final/comments.parquet')
df_news_tweets = pd.read_parquet('../../../covid_media_coverage/data/final/news_tweets.parquet')

In [5]:
df_tweet.shape, df_news_tweets.shape

((17984042, 14), (1814506, 14))

In [6]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(device)

/home/ahaque2/venv/py3/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


cpu


In [7]:
def get_emotion_scores(text, model):

    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt').to(device)
    output = model(**encoded_input)
    scores = output[0][0].cpu().detach().numpy()
    
    #scores = [scores[0], scores[1], scores[3]]
    #label = [labels[0], labels[1], labels[3]]
    scores = softmax(scores)
    
    return scores  #, labels[np.argmax(scores)]

In [11]:
#computed_em_scores = pd.read_csv('results/tweet_emotion_scores.csv')
#l = computed_em_scores.shape[0]

In [8]:
def create_dataframe(df, em_scores):

    em_scores_df = pd.DataFrame()
    #em_scores_df['tweetId'] = df_tweet['tweetId']
    
    df['anger'] = [a[0] for a in em_scores]
    df['joy'] = [a[1] for a in em_scores]
    df['optimism'] = [a[2] for a in em_scores]
    df['sadness'] = [a[3] for a in em_scores]
    
    return df

In [31]:
#em_scores_ = [get_emotion_scores(tweet, model).round(3) for tweet in df_tweet['content'][0:100]]
import sys
import re
em_scores = []

import time
start_time = time.time()
diff = 0
    
x = 50000
i = 0
k = 0
for tweet in df_news_tweets['content'].iloc[k:]:
    
    if(i%x == 0 and i !=0):
        
        df = df_news_tweets.iloc[i-x:i]
        df = create_dataframe(df, em_scores)
        df.to_parquet('covid_data/em_scores/em_scores_' + str(i) + '.parquet')
        
        em_scores = []
    
    if(i%1000 == 0):    
        diff = round((time.time() - start_time) - diff, 3)
        print(i, "\t", diff, end = "\r")
        
    #print(get_emotion_scores(tweet, model).round(3))
    if(len(re.findall(r'[^\w\s,]', tweet)) > 150):
        em_scr = np.array([None, None, None, None])
    else:
        em_scr = get_emotion_scores(tweet, model).round(3)
        
    em_scores.append(em_scr)
    i+=1
    
df = df_news_tweets.iloc[i-x:i]
df = create_dataframe(df, em_scores)
df.to_parquet('covid_data/em_scores/em_scores_' + str(i) + '.parquet')

(14506, 18)243


/home/ahaque2/venv/py3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ahaque2/venv/py3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/ahaque2/venv/py3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [9]:
# df_news_tweet = pd.read_parquet('covid_data/news_tweets_with_em_scores.parquet')
# tmp_df = pd.concat((df_news_tweet, df), axis = 0)
# tmp_df.shape, df_news_tweet.shape
# tmp_df.to_parquet('covid_data/news_tweets_with_em_scores.parquet')